## Computer Vision

Let's do some very basic computer vision. We're going to import the MNIST handwritten digits data and $k$NN to predict values (i.e. "see/read").

1. To load the data, run the following code in a chunk:
```
from keras.datasets import mnist
df = mnist.load_data('minst.db')
train,test = df
X_train, y_train = train
X_test, y_test = test
```
The `y_test` and `y_train` vectors, for each index `i`, tell you want number is written in the corresponding index in `X_train[i]` and `X_test[i]`. The value of `X_train[i]` and `X_test[i]`, however, is a 28$\times$28 array whose entries contain values between 0 and 256. Each element of the matrix is essentially a "pixel" and the matrix encodes a representation of a number. To visualize this, run the following code to see the first ten numbers:
```
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000)
for i in range(5):
    print(y_test[i],'\n') # Print the label
    print(X_test[i],'\n') # Print the matrix of values
    plt.contourf(np.rot90(X_test[i].transpose())) # Make a contour plot of the matrix values
    plt.show()
```
OK, those are the data: Labels attached to handwritten digits encoded as a matrix.

2. What is the shape of `X_train` and `X_test`? What is the shape of `X_train[i]` and `X_test[i]` for each index `i`? What is the shape of `y_train` and `y_test`?
3. Use Numpy's `.reshape()` method to covert the training and testing data from a matrix into an vector of features. So, `X_test[index].reshape((1,784))` will convert the $index$-th element of `X_test` into a $28\times 28=784$-length row vector of values, rather than a matrix. Turn `X_train` into an $N \times 784$ matrix $X$ that is suitable for scikit-learn's kNN classifier where $N$ is the number of observations and $784=28*28$ (you could use, for example, a `for` loop).
4. Use the reshaped `X_test` and `y_test` data to create a $k$-nearest neighbor classifier of digit. What is the optimal number of neighbors $k$? If you can't determine this, play around with different values of $k$ for your classifier.
5. For the optimal number of neighbors, how well does your predictor perform on the test set? Report the accuracy, compute a confusion matrix, and explain your findings.
6. For your confusion matrix, which mistakes are most likely? Do you find any interesting patterns?
7. So, this is how computers "see." They convert an image into a matrix of values, that matrix becomes a vector in a dataset, and then we deploy ML tools on it as if it was any other kind of tabular data. To make sure you follow this, invent a way to represent a color photo in matrix form, and then describe how you could convert it into tabular data. (Hint: RGB color codes provide a method of encoding a numeric value that represents a color.)

In [ ]:
from keras.datasets import mnist
df = mnist.load_data('minst.db')
train,test = df
X_train, y_train = train
X_test, y_test = test

import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000)
for i in range(5):
    print(y_test[i],'\n')
    print(X_test[i],'\n')
    plt.contourf(np.rot90(X_test[i].transpose()))
    plt.show()

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(X_train[0].shape)
print(X_test[0].shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
import pandas as pd
import numpy as np

reload = 0

if reload == 1:
    Z_train = []
    for i in range(60000):
        row = X_train[i].reshape((1,784))
        Z_train.append(row[0])
    Z_train = pd.DataFrame(Z_train)
    Z_train.to_csv('./data/Z_train.csv')
    #
    Z_test = []
    for i in range(len(y_test)):
        row = X_test[i].reshape((1,784))
        Z_test.append(row[0])
    Z_test = pd.DataFrame(Z_test)
    Z_test.to_csv('./data/Z_test.csv')
else:
    Z_train = pd.read_csv('./data/Z_train.csv')
    Z_test = pd.read_csv('./data/Z_test.csv')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

k_bar = 20
accuracy = []

for k in range(k_bar):
    print('Current k: ' , str(k+1))
    knn = KNeighborsClassifier(n_neighbors=k+1)
    predictor = knn.fit(Z_train.values,y_train)
    accuracy.append( predictor.score(Z_test.values,y_test) )

accuracy_max = np.max(accuracy)
max_index = np.where(accuracy==accuracy_max)[0]
k_star = max_index+1
print(k_star)

plt.plot(np.arange(1,k_bar+1),accuracy)
plt.xlabel("k")
plt.title("optimal k: "+str(k_star))
plt.ylabel('Accuracy')
plt.show()

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
predictor = knn.fit(Z_train.values,y_train)
y_hat = predictor.predict(Z_test.values)

accuracy = knn.score(Z_test.values,y_test)
print('Accuracy: ', accuracy)

pd.crosstab(y_test, y_hat)

With \( k=3 \), the rule achieves a 90% accuracy on the test set. When errors occur, they often involve misclassifications such as mistaking 4 for 9, 8 for 3, or 7 for 1, which is reasonable. It is impressive that a straightforward algorithm like kNN can perform so well in classifying such intricate data.

The most frequent misclassifications include confusing a 7 for a 1 (39 times), a 9 for a 7 (40 times), an 8 for a 3 (50 times), a 9 for a 4 (51 times), an 8 for a 5 (45 times), and a 4 for a 9 (81 times). This pattern suggests that these digits share strong visual similarities, making the errors understandable. Even humans occasionally misread these numbers, particularly when they are not clearly written.

The current dataset represents each pixel in the 28×28 grid with an "intensity" value. To incorporate color, we could use three separate matrices to capture the intensity of Red, Green, and Blue channels. These matrices would then be reshaped and combined into a single row, forming tabular data similar to the approach used earlier.